In [ ]:
# extract dataset
from zipfile import ZipFile

dataset_train = r"C:\Users\manasvi patkar\OneDrive\Desktop\Prodigyyyyy\Dataset"
    
with ZipFile(dataset_train, 'r') as zip:
    zip.extractall('Dataset/')

In [ ]:
import os
import numpy as np
import cv2
from zipfile import ZipFile
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Define the path for the zip file
zip_file_path = r"C:\Users\Shraddha\Downloads\archive (12).zip"

# Extract the dataset
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(r"C:\Users\Shraddha\Downloads")  # Adjust this path as needed

# Define paths for training and testing datasets
train_dir = r"C:\Users\Shraddha\Downloads\dataset\training_set"
test_dir = r"C:\Users\Shraddha\Downloads\dataset\test_set"

In [ ]:
# Initialize lists to hold image data and labels
features = []
labels = []
image_size = (50, 50)  # Resize images to this size

# Load and preprocess training images
for category in ['cats', 'dogs']:
    category_path = os.path.join(train_dir, category)
    for image_name in tqdm(os.listdir(category_path), desc=f"Processing {category} Images"):
        image_path = os.path.join(category_path, image_name)
        image_read = cv2.imread(image_path)
        if image_read is None:
            print(f"Error loading image: {image_path}")
            continue
        image_resized = cv2.resize(image_read, image_size)
        image_normalized = image_resized / 255.0  # Normalize the image
        features.append(image_normalized.flatten())  # Flatten the image
        labels.append(0 if category == 'cats' else 1)  # Label cats as 0 and dogs as 1

In [ ]:
# Convert to numpy arrays
features = np.array(features)
labels = np.array(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_pca, y_train)

In [ ]:
y_pred = svm_model.predict(X_test_pca)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Cats', 'Dogs'], yticklabels=['Cats', 'Dogs'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
h = .02

In [ ]:
x_min, x_max = X_train_pca[:, 0].min() - 1, X_train_pca[:, 0].max() + 1
y_min, y_max = X_train_pca[:, 1].min() - 1, X_train_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

In [ ]:
Z = svm_model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

In [ ]:
plt.contourf(xx, yy, Z, alpha=0.8, cmap='coolwarm')
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, edgecolors='k', marker='o', label='Train data')
plt.scatter(X_test_pca[:, 0], X_test_pca[:, 1], c=y_test, edgecolors='k', marker='X', label='Test data')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('SVM Decision Boundary with PCA-reduced Data')
plt.legend()
plt.show()